# Import Package

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPRegressor
from sklearn import metrics


# from sklearn.externals import joblib
import joblib

import pickle
import uuid



# Data

## Download Data

In [2]:
import os.path
if not os.path.isfile('Covid_Data.zip'):
  !gdown --id 1oPQDuqWnH9v72qJiiyqI5LXhdV0yGaPE
  !unzip 'Covid_Data.zip'


/usr/local/lib/python3.8/dist-packages/gdown/cli.py:127: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1oPQDuqWnH9v72qJiiyqI5LXhdV0yGaPE
To: /content/Covid_Data.zip
100% 30.3k/30.3k [00:00<00:00, 27.4MB/s]
Archive:  Covid_Data.zip
   creating: Input Data/
  inflating: Input Data/time_series_19-covid-Confirmed.csv  
  inflating: Input Data/time_series_19-covid-Deaths.csv  
  inflating: Input Data/time_series_19-covid-Recovered.csv  


In [3]:
descriptor = ("d", "r", "c")
filenames = [r'time_series_19-covid-Deaths.csv',
             r'time_series_19-covid-Recovered.csv',
             r'time_series_19-covid-Confirmed.csv']

print(zip(descriptor, filenames))



## Load Data & Feature Enginering;dgk

In [20]:
dfs = []
DATAs = []
for descriptor_, fname in zip(descriptor, filenames):
    print("Working on:",'Input Data/'+ fname)
    df = pd.read_csv('Input Data/'+ fname)
    df.drop(columns=["Province/State","Country/Region"], inplace=True)
    DATA = np.array((0,0,0,0))

    
    for i, j in df.iterrows():
        # print(i, j)
        latitude = j['Lat']
        longitude = j['Long']
        
        for k,l in j.iteritems():
            if k=='Lat':
                continue
            if k=='Long':
                continue
            date = datetime.strptime(k, '%m/%d/%y')
            day = date - date.strptime("01/22/20", '%m/%d/%y')
            days = day.days
            #print(days)
            temp = np.array([j["Lat"], j['Long'], days, l])
            #print(temp)
            DATA = np.vstack((DATA,temp))  
            
    DATA = np.delete(DATA, 0,0)
    np.random.shuffle(DATA)
    
    dfs.append(df)
    DATAs.append(DATA)


Working on: Input Data/time_series_19-covid-Deaths.csv
Working on: Input Data/time_series_19-covid-Recovered.csv
Working on: Input Data/time_series_19-covid-Confirmed.csv


In [15]:
dfs[0]

,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20
0,15.0000,101.0000,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
1,36.0000,138.0000,0,0,0,0,0,0,0,0,...,6,6,6,6,6,6,10,10,15,16
2,1.2833,103.8333,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,28.1667,84.2500,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2.5000,112.5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401,42.5922,-83.3362,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
402,42.2791,-83.3362,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
403,39.5393,-75.6674,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
404,22.0000,-80.0000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
DATAs

[array([[ 30.769 , -86.9824,  45.    ,   0.    ],
        [ 60.    ,  90.    ,  30.    ,   0.    ],
        [ 30.5773, -86.6611,  21.    ,   0.    ],
        ...,
        [ 26.663 , -81.9535,  40.    ,   0.    ],
        [ 22.1667, 113.55  ,   5.    ,   0.    ],
        [  3.2028,  73.2207,  28.    ,   0.    ]]),
 array([[  33.8034,  -84.3963,   47.    ,    0.    ],
        [  45.9432,   24.9668,   49.    ,    6.    ],
        [  35.1917,  108.8701,   39.    ,  208.    ],
        ...,
        [  40.7128,  -74.006 ,    5.    ,    0.    ],
        [  38.4747, -121.3542,    0.    ,    0.    ],
        [  44.7509,  -92.3814,   35.    ,    0.    ]]),
 array([[  41.9029,   12.4534,   34.    ,    0.    ],
        [  27.6104,  111.7088,   34.    , 1016.    ],
        [  32.7732,  -97.3517,   10.    ,    0.    ],
        ...,
        [  18.0708,  -63.0501,   49.    ,    2.    ],
        [  53.9333, -116.5765,   43.    ,    0.    ],
        [  40.7128,  -74.006 ,   36.    ,    0.    ]])]

In [16]:
DATAs[0]

array([[ 30.769 , -86.9824,  45.    ,   0.    ],
       [ 60.    ,  90.    ,  30.    ,   0.    ],
       [ 30.5773, -86.6611,  21.    ,   0.    ],
       ...,
       [ 26.663 , -81.9535,  40.    ,   0.    ],
       [ 22.1667, 113.55  ,   5.    ,   0.    ],
       [  3.2028,  73.2207,  28.    ,   0.    ]])

# Models

In [ ]:
params_dict = [{'hidden_layer_sizes':  [(4,4,4,4),(4,4),(4,4,3,3),(4,3,4),(10,10,10,10,10), (3,), (6,6,6,6), (4,4), (10,5,5,10), (6,), (12,12,12), (3,3,3), (6,6,6), (3,3,3,3,3), (12, 12, 6, 6, 3, 3)],
                'activation': ['relu','identity','logistic','tanh'],
                'solver': ['adam', 'lbfgs'],
                'learning_rate':['constant','adaptive','invscaling'],
                'learning_rate_init': [0.1,0.01,0.5, 0.00001],
                'alpha': [0.01,0.1,0.001, 0.0001],
                'max_iter': [10000]}]


In [26]:
def training(DATA):    
    train_start_time=datetime.now()
    clf = GridSearchCV(MLPRegressor(), params_dict, cv=3, n_jobs=-1, scoring='r2', verbose=10)
    clf.fit(input_train, output_train)
    train_end_time = datetime.now()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    uuid_=uuid.uuid4()
    file = open(descriptor_+"-"+str(uuid_)+"-results.txt", 'w')
    file.write("Data for: "+fname+"\n")
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
          file.write("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params)+"\n")
    file.write("Total training time:"+str(train_end_time-train_start_time)+"\n")
    file.close()
    model_name = descriptor_+"-"+str(uuid_)+".pickle"
    joblib.dump(clf.best_estimator_, model_name)


In [27]:
for DATA in DATAs:
  input_data = DATA[:,:-1]
  print(input_data.shape)
  output_data = DATA[:, -1]
  print(output_data.shape)
  input_train, input_test, output_train, output_test = train_test_split(input_data, output_data)
  training(DATA)

(20706, 3)
(20706,)
Fitting 3 folds for each of 5760 candidates, totalling 17280 fits


KeyboardInterrupt: ignored

In [ ]:
# for descriptor_, fname in zip(descriptor, filenames):
#     print("Working on:",'Input Data/'+ fname)
#     df = pd.read_csv('Input Data/'+ fname)
#     df.drop(columns=["Province/State","Country/Region"], inplace=True)
    
#     DATA = np.array((0,0,0,0))
    
#     for i, j in df.iterrows():
#         #print(i, j)
#         latitude = j['Lat']
#         longitude = j['Long']
        
#         for k,l in j.iteritems():
#             if k=='Lat':
#                 continue
#             if k=='Long':
#                 continue
#             date = datetime.strptime(k, '%m/%d/%y')
#             day = date - date.strptime("01/22/20", '%m/%d/%y')
#             days = day.days
#             #print(days)
#             temp = np.array([j["Lat"], j['Long'], days, l])
#             #print(temp)
#             DATA = np.vstack((DATA,temp))  
            
#     DATA = np.delete(DATA, 0,0)
#     np.random.shuffle(DATA)
    
#     input_data = DATA[:,:-1]
#     output_data = DATA[:, -1]
    
#     input_train, input_test, output_train, output_test = train_test_split(input_data, output_data)
    
    
#     train_start_time=datetime.now()
#     clf = GridSearchCV(MLPRegressor(), params_dict, cv=3, n_jobs=-1, scoring='r2', verbose=10)
#     clf.fit(input_train, output_train)
#     train_end_time = datetime.now()
#     means = clf.cv_results_['mean_test_score']
#     stds = clf.cv_results_['std_test_score']
#     uuid_=uuid.uuid4()
#     file = open(descriptor_+"-"+str(uuid_)+"-results.txt", 'w')
#     file.write("Data for: "+fname+"\n")
#     for mean, std, params in zip(means, stds, clf.cv_results_['params']):
          
#           file.write("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params)+"\n")
#     file.write("Total training time:"+str(train_end_time-train_start_time)+"\n")
#     file.close()
#     model_name = descriptor_+"-"+str(uuid_)+".pickle"
#     joblib.dump(clf.best_estimator_, model_name)


Working on: Input Data/time_series_19-covid-Deaths.csv
Fitting 3 folds for each of 5760 candidates, totalling 17280 fits
